In [7]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
import json
import pandas as pd
import datetime
import pytz
import talib

API_KEY = "I0D0ldkssJgACZm3fcs9DlZAX5osmgZp"

from utils import grab_new_data_polygon
from indicators_lib import *
from preprocess import parse_condition
from indicators import extract_indicators

In [18]:
dummy_cond = "rsi(period=14)[-1]>sma(period=50)[-1]"

In [ ]:
(parse_condition(dummy_cond)[0])

[[['rsi', 'period=14', '[-1]'], '>', ['sma', 'period=50', '[-1]']]]

In [19]:
extract_indicators(parse_condition(dummy_cond)[0])

{('rsi', 'period=14'), ('sma', 'period=50')}